# [Distributed] Pre-training and fine-tuning an LLM on CPU on AG News with ThirdAI's UDT

This training script shows how to run data parallel pre-training of an LLM from scratch on the popular AG News Dataset (https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset) using ThirdAI's Universal Deep Transformer (UDT).

## 1. Import thirdai and activate license

In [1]:
!pip3 install datasets
!pip3 install thirdai --upgrade
!pip3 install ray
!pip3 install torch

import thirdai
from thirdai import bolt
import os
import thirdai.distributed_bolt as dist  
thirdai.licensing.activate('XFTR-LPUL-HFRY-V4A3-NAR4-YEUK-X9VT-MNLR')

/bin/pip3:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
/bin/pip3:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
Requirement already up-to-date: thirdai in /home/pratik/.local/lib/python3.8/site-packages (0.7.16)
/bin/pip3:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
/bin/pip3:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


/home/pratik/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-18 08:44:45,131	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-18 08:44:46,417	WARNING collective.py:20 -- NCCL seems unavailable. Please install Cupy following the guide at: https://docs.cupy.dev/en/stable/install.html.
2023-08-18 08:44:46,570	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-18 08:44:46,825	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


RuntimeError: The license was found to be invalid: The licensing server says that your license is invalid. It returned the following message: does not exist

## 2. Ray Cluster Initialization
 For the purpose of this demo, we will be initializing a mock ray cluster of 2 nodes here. Change num_cpus accordingly.

In [ ]:
import ray
from ray.air import ScalingConfig, session

cpus_per_node = (dist.get_num_cpus() - 1) // 2

ray.init(ignore_reinit_error=True, runtime_env={"env_vars": {"OMP_NUM_THREADS": f"{cpus_per_node}"}})
scaling_config = ScalingConfig(
    num_workers=2,
    use_gpu=False,
    trainer_resources={"CPU": 1},
    resources_per_worker={"CPU": cpus_per_node},
    placement_strategy="PACK",
)

2023-08-18 07:32:03,745	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


## 3. Download and process the dataset into a csv file. <br>
### 3.1 We divide our dataset into 2 datasets for purpose of data-parallel training. 

In [ ]:
from datasets import load_dataset

file_1 = open('agnews_train_0.csv', 'w')
file_2 = open('agnews_train_1.csv', 'w')

corpus = load_dataset("ag_news")["train"]["text"]
num_datapoints = len(corpus)

file_1.write("id,text\n")
file_2.write("id,text\n")

idx = 0
for line in corpus:
    if idx < num_datapoints//2:
        nothing = file_1.write(str(idx) + "," + line.replace(",", " ").lower() + "\n")
    else:
        nothing = file_2.write(str(idx) + "," + line.replace(",", " ").lower() + "\n")

    idx += 1

file_1.close()
file_2.close()

train_filenames = ['agnews_train_0.csv', 'agnews_train_1.csv']

Found cached dataset ag_news (/home/pratik/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|██████████| 2/2 [00:00<00:00, 469.16it/s]


### 3.2 Looking at the dataset
In the above step, *agnews_train_[0/1].csv* files refers to the corpus file with document id and text. We can have even more columns with other metadata for each row. <br>

A couple of sample rows of the *corpus_file* are shown below.

In [ ]:
import pandas as pd

pd.options.display.max_colwidth = 700
pd.read_csv(train_filenames[0], nrows=4)

,id,text
0,0,wall st. bears claw back into the black (reuters) reuters - short-sellers wall street's dwindling\band of ultra-cynics are seeing green again.
1,1,carlyle looks toward commercial aerospace (reuters) reuters - private investment firm carlyle group \which has a reputation for making well-timed and occasionally\controversial plays in the defense industry has quietly placed\its bets on another part of the market.
2,2,oil and economy cloud stocks' outlook (reuters) reuters - soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
3,3,iraq halts oil exports from main southern pipeline (reuters) reuters - authorities have halted oil export\flows from the main pipeline in southern iraq after\intelligence showed a rebel militia could strike\infrastructure an oil official said on saturday.


## 4. Define a UDT model and training loop

In the UDT model, <i>data_type</i> *query* can be anything of your choice but *id* should match with the one in the header of the *corpus_file*.

The <b><i>train_loop_per_worker</i></b> defines the training that will run on each worker node.

Pre-training with UDT supports two types of columns, strong and weak. For the purpose of this demo, we choose *text* to be the strong column and leave the weak column list to be empty.<br>
We will now train a UDT model in distributed data parallel fashion. Feel free to customize the number of epochs and the learning rate. <br>
<u><b>PLEASE NOTE :</b></u> Currently, UDT's cold_start function requires the *id* to be an integer. We will add support for other formats in a future release.

In [ ]:
def get_udt_model(model_config_path):
    model = bolt.UniversalDeepTransformer(
        data_types={
            "query": bolt.types.text(),
            "id": bolt.types.categorical(delimiter=':'),
        },
        target="id",
        n_target_classes=num_datapoints,
        integer_target=True,
        model_config=model_config_path,
    )
    return model
    
def train_loop_per_worker(config):
    # thirdai.licensing.deactivate()
    # thirdai.licensing.activate("HN7J-W79C-KN9U-WTKE-9PNM-4PVR-CNPJ-WTWE")
    thirdai.logging.setup(log_to_stderr=False, path="log.txt", level="info")
    thirdai.licensing.activate('XFTR-LPUL-HFRY-V4A3-NAR4-YEUK-X9VT-MNLR')
    
    
    model_config_path = os.path.join(config["curr_dir"], '../configs/embeddings_and_cold_start_0.005.config')
    model = get_udt_model(model_config_path)
    model = dist.prepare_model(model)

    metrics = model.coldstart_distributed_v2(
        filename=os.path.join(config["curr_dir"], train_filenames[session.get_world_rank()]),
        strong_column_names=["text"],
        weak_column_names=[],
        learning_rate=0.001,
        epochs=5,
        metrics=["categorical_accuracy"],
        verbose=True,
    )

    session.report(
        metrics=metrics,
        checkpoint=dist.UDTCheckPoint.from_model(model),
    )


## 5. Distributed Training

Now, we start the training using <b>ThirdAI</b> <i>BoltTrainer</i> which runs the <i>train_loop_per_worker</i> function on different worker nodes.

In [ ]:
import os
from ray.train.torch import TorchConfig

trainer = dist.BoltTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={
        "curr_dir": os.path.abspath(os.getcwd()),
    },
    scaling_config=scaling_config,
    backend_config=TorchConfig(backend="gloo"),
)

result_checkpoint_and_history = trainer.fit()

(pid=2996037) NCCL seems unavailable. Please install Cupy following the guide at: https://docs.cupy.dev/en/stable/install.html.
(BoltTrainer pid=2996037) Starting distributed worker processes: ['2996143 (192.168.1.10)', '2996144 (192.168.1.10)']
(RayTrainWorker pid=2996143) Setting up process group for: env:// [rank=0, world_size=2]
2023-08-18 07:32:31,484	ERROR tune_controller.py:911 -- Trial task failed for trial BoltTrainer_56884_00000
Traceback (most recent call last):
  File "/home/pratik/.local/lib/python3.8/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/pratik/.local/lib/python3.8/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/pratik/.local/lib/python3.8/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/pratik/.local/lib/python3.8/site-packages/ray/_privat

TrainingFailedError: The Ray Train run failed. Please inspect the previous error messages for a cause. After fixing the issue (assuming that the error is not caused by your own application logic, but rather an error such as OOM), you can restart the run from scratch or continue this run.
To continue this run, you can use: `trainer = BoltTrainer.restore("/home/pratik/ray_results/BoltTrainer_2023-08-18_07-32-08")`.
To start a new run that will retry on training failures, set `air.RunConfig(failure_config=air.FailureConfig(max_failures))` in the Trainer's `run_config` with `max_failures > 0`, or `max_failures = -1` for unlimited retries.

## 6. Save and load the model

In [ ]:
model = result_checkpoint_and_history.checkpoint.get_model()
model.save('./agnews.model')

model = bolt.UniversalDeepTransformer.load('./agnews.model')

## 7. Make Predictions

### Example 1

In [ ]:
import numpy as np
import pandas as pd

from thirdai.demos import download_agnews_dataset

corpus_file = './agnews.csv'
download_agnews_dataset(corpus_file)

df = pd.read_csv(corpus_file)

activations = model.predict({'query':'BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases'})
top_preds = np.argsort(-activations)[:5]

df.iloc[top_preds]

For the same example, here are the top-5 results that OpenAI's Search and Recommendation notebook (https://github.com/openai/openai-cookbook/blob/main/examples/Recommendation_using_embeddings.ipynb) gets.

| text |
| --- |
| THE re-election of British Prime Minister Tony Blair would be seen as an endorsement of the military action in Iraq, Prime Minister John Howard said today |
| LONDON, England -- A US scientist is reported to have observed a surprising jump in the amount of carbon dioxide, the main greenhouse gas. |
| The anguish of hostage Kenneth Bigley in Iraq hangs over Prime Minister Tony Blair today as he faces the twin test of a local election and a debate by his Labour Party about the divisive war. |
| Israel is prepared to back a Middle East conference convened by Tony Blair early next year despite having expressed fears that the British plans were over-ambitious and designed |
| AFP - A battle group of British troops rolled out of southern Iraq on a US-requested mission to deadlier areas near Baghdad, in a major political gamble for British Prime Minister Tony Blair. |


### Example 2

In [ ]:
activations = model.predict({'query':'PC World - Upcoming chip set will include built-in security features for your PC'})
top_preds = np.argsort(-activations)[:5]

df.iloc[top_preds]

For the same example, here are the top-5 results that OpenAI's Search and Recommendation notebook (https://github.com/openai/openai-cookbook/blob/main/examples/Recommendation_using_embeddings.ipynb) gets.

| text |
| --- |
| PC World - Updated antivirus software for businesses adds intrusion prevention features. |
| PC World - The one-time World Class Product of the Year PDA gets a much-needed upgrade. |
| PC World - Send your video throughout your house--wirelessly--with new gateways and media adapters. |
| PC World - Symantec, McAfee hope raising virus-definition fees will move users to\  suites. |
| Gateway computers will be more widely available at Office Depot, in the PC maker #39;s latest move to broaden distribution at retail stores since acquiring rival eMachines this year. |

# Ray Cluster Teardown

In [ ]:
ray.shutdown()